In [ ]:
# Example 15: relativistic correction due to elliptic orbit
# SPDX-FileCopyrightText: Copyright (C) 2023 Andreas Naber <annappo@web.de>
# SPDX-License-Identifier: GPL-3.0-only

%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np

G = 6.6743015E-11               # gravitational constant
ME = 5.9722E24                  # earth mass
REP = 6356752                   # earth radius to pole, WGS84
GPS_C  = 2.99792458e8           # speed of light as defined for GPS
F = -2*np.sqrt(G*ME)/GPS_C**2   # GPS -4.44280763E-10
a = 26600000                    # major axis
N = 10000                       # number of point for integration and plot 
e = 0.01                        # eccentricity
b = a*np.sqrt(1-e**2)           # minor axis; e*a = sqrt(a**-b**2)

TE = 2*np.pi*np.sqrt(a**3/(G*ME))   # time period

# Kepler equation: M=E-e*sin(E) with M mean anomaly and E eccentric anomaly
def kepler(M,e):
    itMax=10
    eps=1.0E-12
    E = []
    for M_k in M:
        E_k = M_k        
        it = 0
        q0 = 0
        while abs(E_k-q0) > eps and it < itMax:
            q0 = E_k
            E_k = q0-(q0-e*np.sin(q0)-M_k)/(1-e*np.cos(q0))            
            it += 1            
        E.append(E_k)
    E = np.asarray(E)
    return E

M = np.linspace(0,2*np.pi,N)               # mean anomaly
t = M/(2*np.pi)*TE                         # time

E = kepler(M,e)                            # eccentric anomaly

x = a*np.cos(E) - a*e                      # used for plot
y = b*np.sin(E)                            #      "
r = np.sqrt(x**2 + y**2)                   # distance to earth
# as an alternative you can use:
# r = a*(1-e*np.cos(E))

Phi = -G*ME/r                              # potential
v = np.sqrt(G*ME*(2/r - 1/a))              # velocity

gammaS = 1 - v**2/(2*GPS_C**2)             # special relativity
gammaG = 1 + Phi/GPS_C**2                  # general relativity, only potential
dtr = (gammaS + gammaG - 2)                # correction for clock on satellite
dtrMean = np.mean(dtr)
dtrTheory = np.cumsum(dtr-dtrMean)*t[1]    # relativistic correction 
dtrGPS = F*e*np.sqrt(a)*np.sin(E)          # Replace here E with M to see 
                                           # differences (see text)

gammaOnEarth = 1 - G*ME/(REP*GPS_C**2)     # equal at earth surface due
dtrEarth = gammaOnEarth-1                  # to centrifugal potential 

fig,(ax,bx) = plt.subplots(1,2,figsize=(8,3))
fig.canvas.header_visible = False
ax.set_aspect('equal')
ax.plot(x/a,y/a)
ax.grid()
ax.set_xlabel('x / a')
ax.set_ylabel('y / a')

bx.plot(t/3600,dtrTheory/1E-9,lw=3,label='Theory e=%1.2f' % (e))
bx.plot(t/3600,dtrGPS/1E-9,lw=1,label='GPS e=%1.2f' % (e))
bx.set_xlabel('time / h')
bx.set_ylabel('relativistic correction / ns')
bx.grid()
bx.legend()

plt.tight_layout()
plt.show()
